<a href="https://colab.research.google.com/github/thomazcampos07/hello-world/blob/main/Assignment1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This assignment is part I of your evaluation process. You should address every question of the assignment on the cell below. If you decide to write any explanation, you should do so commenting on your code cells or by creating a new markdown cell.

**Name:** Thomaz Daniel Moreira Campos

**CPF:** 008.138.101-83  

**Date:** 24/01/21

# Assignment 1 - Data Wrangling Skills

### Question 1 
Load the energy data from the file `Energy Indicators.xls`, which is a list of indicators of [energy supply and renewable electricity production](Energy%20Indicators.xls) from the [United Nations](http://unstats.un.org/unsd/environment/excel_file_tables/2013/Energy%20Indicators.xls) for the year 2013, and should be put into a DataFrame with the variable name of **energy**.

Keep in mind that this is an Excel file, and not a comma separated values file. Also, make sure to exclude the footer and header information from the datafile. The first two columns are unneccessary, so you should get rid of them, and you should change the column labels so that the columns are:

`['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable']`

Convert `Energy Supply` to gigajoules (there are 1,000,000 gigajoules in a petajoule). For all countries which have missing data (e.g. data with "...") make sure this is reflected as `np.NaN` values.

Rename the following list of countries (for use in later questions):

```"Republic of Korea": "South Korea",
"United States of America": "United States",
"United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
"China, Hong Kong Special Administrative Region": "Hong Kong"```

There are also several countries with numbers and/or parenthesis in their name. Be sure to remove these, 

e.g. 

`'Bolivia (Plurinational State of)'` should be `'Bolivia'`, 

`'Switzerland17'` should be `'Switzerland'`.

<br>

Next, load the GDP data from the file `world_bank.csv`, which is a csv containing countries' GDP from 1960 to 2015 from [World Bank](http://data.worldbank.org/indicator/NY.GDP.MKTP.CD). Call this DataFrame **GDP**. 

Make sure to skip the header, and rename the following list of countries:

```"Korea, Rep.": "South Korea", 
"Iran, Islamic Rep.": "Iran",
"Hong Kong SAR, China": "Hong Kong"```

<br>

Finally, load the [Sciamgo Journal and Country Rank data for Energy Engineering and Power Technology](http://www.scimagojr.com/countryrank.php?category=2102) from the file `scimagojr-3.xlsx`, which ranks countries based on their journal contributions in the aforementioned area. Call this DataFrame **ScimEn**.

Join the three datasets: GDP, Energy, and ScimEn into a new dataset (using the intersection of country names). Use only the last 10 years (2006-2015) of GDP data and only the top 15 countries by Scimagojr 'Rank' (Rank 1 through 15). 

The index of this DataFrame should be the name of the country, and the columns should be ['Rank', 'Documents', 'Citable documents', 'Citations', 'Self-citations',
       'Citations per document', 'H index', 'Energy Supply',
       'Energy Supply per Capita', '% Renewable', '2006', '2007', '2008',
       '2009', '2010', '2011', '2012', '2013', '2014', '2015'].

*This function should return a DataFrame with 20 columns and 15 entries.*

In [ ]:
def answer_one():
    import pandas as pd
    import numpy as np
    import math as mt

    #pd.set_option('max_rows', 99999)
    #pd.set_option('max_colwidth', 99999)
    #pd.describe_option('max_colwidth')

    #ler arquivo Energy Indicators.xls, já excluindo cabeçalho e rodapé
    energy = pd.read_excel("Energy_Indicators.xls", sheet_name="Energy", skiprows = 16, nrows = 228)
    
    #excluir primeiras colunas
    energy = energy.rename(columns = {'Unnamed: 2':'Country'})
    energy.drop(energy.columns[energy.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)

    #renomear colunas
    energy.columns = ['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable']

    #converter Energy Supply para gigajoules
    energy['Energy Supply'] = energy['Energy Supply'].apply(pd.to_numeric, errors='coerce')
    energy['Energy Supply'] = energy['Energy Supply']*1000000

    #renomear países
    energy['Country'] = energy['Country'].str.replace('\d+', '')
    energy = energy.replace(to_replace = 'Republic of Korea', value = 'South Korea')
    energy = energy.replace(to_replace = 'United States of America', value = 'United States')
    energy = energy.replace(to_replace = 'United Kingdom of Great Britain and Northern Ireland', value = 'United Kingdom')
    energy = energy.replace(to_replace = 'China, Hong Kong Special Administrative Region', value = 'Hong Kong')
    energy = energy.replace(to_replace = 'Bolivia (Plurinational State of)', value = 'Bolivia')
    energy = energy.replace(to_replace = 'Falkland Islands (Malvinas)', value = 'Falkland Islands')
    energy = energy.replace(to_replace = 'Iran (Islamic Republic of)', value = 'Iran')
    energy = energy.replace(to_replace = 'Micronesia (Federated States of)', value = 'Micronesia')
    energy = energy.replace(to_replace = 'Sint Maarten (Dutch part)', value = 'Sint Maarten')
    energy = energy.replace(to_replace = 'Venezuela (Bolivarian Republic of)', value = 'Venezuela')

    #ler arquivo world_bank.csv, já excluindo cabeçalho
    GDP = pd.read_csv('world_bank.csv', sep=",", header = 4)

    #renomear países
    GDP = GDP.replace(to_replace = 'Korea, Rep.', value = 'South Korea')
    GDP = GDP.replace(to_replace = 'Iran, Islamic Rep.', value = 'Iran')
    GDP = GDP.replace(to_replace = 'Hong Kong SAR, China', value = 'Hong Kong')

    #ler arquivo scimagojr-3.xlsx
    ScimEn = pd.read_excel('scimagojr-3.xlsx')

    #GDP entre 2006 e 2015
    GDP = GDP[['Country Name', 'Country Code', 'Indicator Name', 'Indicator Code'
        ,'2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015']]

    #mesclar
    base = pd.merge(ScimEn, energy, how = "left", left_on = "Country", right_on = "Country", indicator = False)
    base = pd.merge(base, GDP, how = "left", left_on = "Country", right_on = "Country Name", indicator = False)

    #somente os 15 primeiros
    base = base[0:15]

    #index e escolher colunas
    base = base.set_index(['Country'])
    base = base[['Rank', 'Documents', 'Citable documents', 'Citations', 'Self-citations'
    , 'Citations per document', 'H index', 'Energy Supply', 'Energy Supply per Capita'
    , '% Renewable', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015']]
    
    return base


In [ ]:
answer_one()

,Rank,Documents,Citable documents,Citations,Self-citations,Citations per document,H index,Energy Supply,Energy Supply per Capita,% Renewable,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
Country,,,,,,,,,,,,,,,,,,,,
China,1,127050,126767,597237,411683,4.70,138,1.271910e+11,93,19.7549,3.992331e+12,4.559041e+12,4.997775e+12,5.459247e+12,6.039659e+12,6.612490e+12,7.124978e+12,7.672448e+12,8.230121e+12,8.797999e+12
United States,2,96661,94747,792274,265436,8.20,230,9.083800e+10,286,11.571,1.479230e+13,1.505540e+13,1.501149e+13,1.459484e+13,1.496437e+13,1.520402e+13,1.554216e+13,1.577367e+13,1.615662e+13,1.654857e+13
Japan,3,30504,30287,223024,61554,7.31,134,1.898400e+10,149,10.2328,5.496542e+12,5.617036e+12,5.558527e+12,5.251308e+12,5.498718e+12,5.473738e+12,5.569102e+12,5.644659e+12,5.642884e+12,5.669563e+12
United Kingdom,4,20944,20357,206091,37874,9.84,139,7.920000e+09,124,10.6005,2.419631e+12,2.482203e+12,2.470614e+12,2.367048e+12,2.403504e+12,2.450911e+12,2.479809e+12,2.533370e+12,2.605643e+12,2.666333e+12
Russian Federation,5,18534,18301,34266,12422,1.85,57,3.070900e+10,214,17.2887,1.385793e+12,1.504071e+12,1.583004e+12,1.459199e+12,1.524917e+12,1.589943e+12,1.645876e+12,1.666934e+12,1.678709e+12,1.616149e+12
Canada,6,17899,17620,215003,40930,12.01,149,1.043100e+10,296,61.9454,1.564469e+12,1.596740e+12,1.612713e+12,1.565145e+12,1.613406e+12,1.664087e+12,1.693133e+12,1.730688e+12,1.773486e+12,1.792609e+12
Germany,7,17027,16831,140566,27426,8.26,126,1.326100e+10,165,17.9015,3.332891e+12,3.441561e+12,3.478809e+12,3.283340e+12,3.417298e+12,3.542371e+12,3.556724e+12,3.567317e+12,3.624386e+12,3.685556e+12
India,8,15005,14841,128763,37209,8.58,115,3.319500e+10,26,14.9691,1.265894e+12,1.374865e+12,1.428361e+12,1.549483e+12,1.708459e+12,1.821872e+12,1.924235e+12,2.051982e+12,2.200617e+12,2.367206e+12
France,9,13153,12973,130632,28601,9.93,114,1.059700e+10,166,17.0203,2.607840e+12,2.669424e+12,2.674637e+12,2.595967e+12,2.646995e+12,2.702032e+12,2.706968e+12,2.722567e+12,2.729632e+12,2.761185e+12


### Question 2
The previous question joined three datasets then reduced this to just the top 15 entries. When you joined the datasets, but before you reduced this to the top 15 items, how many entries did you lose?

*This function should return a single number.*

In [ ]:
%%HTML
<svg width="800" height="300">
  <circle cx="150" cy="180" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="blue" />
  <circle cx="200" cy="100" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="red" />
  <circle cx="100" cy="100" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="green" />
  <line x1="150" y1="125" x2="300" y2="150" stroke="black" stroke-width="2" fill="black" stroke-dasharray="5,3"/>
  <text  x="300" y="165" font-family="Verdana" font-size="35">Everything but this!</text>
</svg>

In [ ]:
def answer_two():
    import pandas as pd
    import numpy as np
    import math as mt

    #ler arquivo Energy Indicators.xls, já excluindo cabeçalho e rodapé
    energy = pd.read_excel("Energy_Indicators.xls", sheet_name="Energy", skiprows = 16, nrows = 228)
    
    #excluir primeiras colunas
    energy = energy.rename(columns = {'Unnamed: 2':'Country'})
    energy.drop(energy.columns[energy.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)

    #renomear colunas
    energy.columns = ['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable']

    #converter Energy Supply para gigajoules
    energy['Energy Supply'] = energy['Energy Supply'].apply(pd.to_numeric, errors='coerce')
    energy['Energy Supply'] = energy['Energy Supply']*1000000

    #renomear países
    energy['Country'] = energy['Country'].str.replace('\d+', '')
    energy = energy.replace(to_replace = 'Republic of Korea', value = 'South Korea')
    energy = energy.replace(to_replace = 'United States of America', value = 'United States')
    energy = energy.replace(to_replace = 'United Kingdom of Great Britain and Northern Ireland', value = 'United Kingdom')
    energy = energy.replace(to_replace = 'China, Hong Kong Special Administrative Region', value = 'Hong Kong')
    energy = energy.replace(to_replace = 'Bolivia (Plurinational State of)', value = 'Bolivia')
    energy = energy.replace(to_replace = 'Falkland Islands (Malvinas)', value = 'Falkland Islands')
    energy = energy.replace(to_replace = 'Iran (Islamic Republic of)', value = 'Iran')
    energy = energy.replace(to_replace = 'Micronesia (Federated States of)', value = 'Micronesia')
    energy = energy.replace(to_replace = 'Sint Maarten (Dutch part)', value = 'Sint Maarten')
    energy = energy.replace(to_replace = 'Venezuela (Bolivarian Republic of)', value = 'Venezuela')

    #ler arquivo world_bank.csv, já excluindo cabeçalho
    GDP = pd.read_csv('world_bank.csv', sep=",", header = 4)

    #renomear países
    GDP = GDP.replace(to_replace = 'Korea, Rep.', value = 'South Korea')
    GDP = GDP.replace(to_replace = 'Iran, Islamic Rep.', value = 'Iran')
    GDP = GDP.replace(to_replace = 'Hong Kong SAR, China', value = 'Hong Kong')

    #ler arquivo scimagojr-3.xlsx
    ScimEn = pd.read_excel('scimagojr-3.xlsx')

    #somente os 15 primeiros países
    #ScimEn = ScimEn[0:15]

    #GDP entre 2006 e 2015
    GDP = GDP[['Country Name', 'Country Code', 'Indicator Name', 'Indicator Code'
        ,'2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015']]

    #mesclar
    base = pd.merge(ScimEn, energy, how = "outer", left_on = "Country", right_on = "Country", indicator = True)
    base = base.rename(columns = {'_merge':'merge1'})
    base = pd.merge(base, GDP, how = "outer", left_on = "Country", right_on = "Country Name", indicator = True)
    base = base.rename(columns = {'_merge':'merge2'})

    linhas = len(base.index)
    linhas = linhas - 15

    return linhas

  

In [ ]:
answer_two()

304


## Answer the following questions in the context of only the top 15 countries by Scimagojr Rank (aka the DataFrame returned by `answer_one()`)

### Question 3
What is the average GDP over the last 10 years for each country? (exclude missing values from this calculation.)

*This function should return a Series named `avgGDP` with 15 countries and their average GDP sorted in descending order.*

In [ ]:
def answer_three():

    import pandas as pd
    import numpy as np
    import math as mt

    #pd.set_option('max_rows', 99999)
    #pd.set_option('max_colwidth', 99999)
    #pd.describe_option('max_colwidth')

    #ler arquivo Energy Indicators.xls, já excluindo cabeçalho e rodapé
    energy = pd.read_excel("Energy_Indicators.xls", sheet_name="Energy", skiprows = 16, nrows = 228)
    
    #excluir primeiras colunas
    energy = energy.rename(columns = {'Unnamed: 2':'Country'})
    energy.drop(energy.columns[energy.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)

    #renomear colunas
    energy.columns = ['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable']

    #converter Energy Supply para gigajoules
    energy['Energy Supply'] = energy['Energy Supply'].apply(pd.to_numeric, errors='coerce')
    energy['Energy Supply'] = energy['Energy Supply']*1000000

    #renomear países
    energy['Country'] = energy['Country'].str.replace('\d+', '')
    energy = energy.replace(to_replace = 'Republic of Korea', value = 'South Korea')
    energy = energy.replace(to_replace = 'United States of America', value = 'United States')
    energy = energy.replace(to_replace = 'United Kingdom of Great Britain and Northern Ireland', value = 'United Kingdom')
    energy = energy.replace(to_replace = 'China, Hong Kong Special Administrative Region', value = 'Hong Kong')
    energy = energy.replace(to_replace = 'Bolivia (Plurinational State of)', value = 'Bolivia')
    energy = energy.replace(to_replace = 'Falkland Islands (Malvinas)', value = 'Falkland Islands')
    energy = energy.replace(to_replace = 'Iran (Islamic Republic of)', value = 'Iran')
    energy = energy.replace(to_replace = 'Micronesia (Federated States of)', value = 'Micronesia')
    energy = energy.replace(to_replace = 'Sint Maarten (Dutch part)', value = 'Sint Maarten')
    energy = energy.replace(to_replace = 'Venezuela (Bolivarian Republic of)', value = 'Venezuela')

    #ler arquivo world_bank.csv, já excluindo cabeçalho
    GDP = pd.read_csv('world_bank.csv', sep=",", header = 4)

    #renomear países
    GDP = GDP.replace(to_replace = 'Korea, Rep.', value = 'South Korea')
    GDP = GDP.replace(to_replace = 'Iran, Islamic Rep.', value = 'Iran')
    GDP = GDP.replace(to_replace = 'Hong Kong SAR, China', value = 'Hong Kong')

    #ler arquivo scimagojr-3.xlsx
    ScimEn = pd.read_excel('scimagojr-3.xlsx')

    #somente os 15 primeiros países
    #ScimEn = ScimEn[0:15]

    #GDP entre 2006 e 2015
    GDP = GDP[['Country Name', 'Country Code', 'Indicator Name', 'Indicator Code'
        ,'2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015']]

    #mesclar
    base = pd.merge(ScimEn, energy, how = "left", left_on = "Country", right_on = "Country", indicator = False)
    base = pd.merge(base, GDP, how = "left", left_on = "Country", right_on = "Country Name", indicator = False)

    #somente os 15 primeiros
    base = base[0:15]

    #index e escolher colunas
    #base = base.set_index(['Country'])
    base = base[['Country','Rank', 'Documents', 'Citable documents', 'Citations', 'Self-citations'
    , 'Citations per document', 'H index', 'Energy Supply', 'Energy Supply per Capita'
    , '% Renewable', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015']]
    
    base['avgGDP'] = base[['2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015']].mean(axis=1)
    #base = base['avgGDP']
    base = base.sort_values(by= ['avgGDP'], ascending = False)

    #converter para série
    serie = pd.Series(base['avgGDP'].values, index = base['Country'], name = 'avgGDP')

    return serie

In [ ]:
answer_three()

Country
United States         1.536434e+13
China                 6.348609e+12
Japan                 5.542208e+12
Germany               3.493025e+12
France                2.681725e+12
United Kingdom        2.487907e+12
Brazil                2.189794e+12
Italy                 2.120175e+12
India                 1.769297e+12
Canada                1.660647e+12
Russian Federation    1.565459e+12
Spain                 1.418078e+12
Australia             1.164043e+12
South Korea           1.106715e+12
Iran                  4.441558e+11
Name: avgGDP, dtype: float64


### Question 4
By how much had the GDP changed over the 10 year span for the country with the 6th largest average GDP?

*This function should return a single number.*

In [ ]:
def answer_four():

    import pandas as pd
    import numpy as np
    import math as mt

    #pd.set_option('max_rows', 99999)
    #pd.set_option('max_colwidth', 99999)
    #pd.describe_option('max_colwidth')

    #ler arquivo Energy Indicators.xls, já excluindo cabeçalho e rodapé
    energy = pd.read_excel("Energy_Indicators.xls", sheet_name="Energy", skiprows = 16, nrows = 228)
    
    #excluir primeiras colunas
    energy = energy.rename(columns = {'Unnamed: 2':'Country'})
    energy.drop(energy.columns[energy.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)

    #renomear colunas
    energy.columns = ['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable']

    #converter Energy Supply para gigajoules
    energy['Energy Supply'] = energy['Energy Supply'].apply(pd.to_numeric, errors='coerce')
    energy['Energy Supply'] = energy['Energy Supply']*1000000

    #renomear países
    energy['Country'] = energy['Country'].str.replace('\d+', '')
    energy = energy.replace(to_replace = 'Republic of Korea', value = 'South Korea')
    energy = energy.replace(to_replace = 'United States of America', value = 'United States')
    energy = energy.replace(to_replace = 'United Kingdom of Great Britain and Northern Ireland', value = 'United Kingdom')
    energy = energy.replace(to_replace = 'China, Hong Kong Special Administrative Region', value = 'Hong Kong')
    energy = energy.replace(to_replace = 'Bolivia (Plurinational State of)', value = 'Bolivia')
    energy = energy.replace(to_replace = 'Falkland Islands (Malvinas)', value = 'Falkland Islands')
    energy = energy.replace(to_replace = 'Iran (Islamic Republic of)', value = 'Iran')
    energy = energy.replace(to_replace = 'Micronesia (Federated States of)', value = 'Micronesia')
    energy = energy.replace(to_replace = 'Sint Maarten (Dutch part)', value = 'Sint Maarten')
    energy = energy.replace(to_replace = 'Venezuela (Bolivarian Republic of)', value = 'Venezuela')

    #ler arquivo world_bank.csv, já excluindo cabeçalho
    GDP = pd.read_csv('world_bank.csv', sep=",", header = 4)

    #renomear países
    GDP = GDP.replace(to_replace = 'Korea, Rep.', value = 'South Korea')
    GDP = GDP.replace(to_replace = 'Iran, Islamic Rep.', value = 'Iran')
    GDP = GDP.replace(to_replace = 'Hong Kong SAR, China', value = 'Hong Kong')

    #ler arquivo scimagojr-3.xlsx
    ScimEn = pd.read_excel('scimagojr-3.xlsx')

    #somente os 15 primeiros países
    #ScimEn = ScimEn[0:15]

    #GDP entre 2006 e 2015
    GDP = GDP[['Country Name', 'Country Code', 'Indicator Name', 'Indicator Code'
        ,'2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015']]

    #mesclar
    base = pd.merge(ScimEn, energy, how = "left", left_on = "Country", right_on = "Country", indicator = False)
    base = pd.merge(base, GDP, how = "left", left_on = "Country", right_on = "Country Name", indicator = False)

    #somente os 15 primeiros
    base = base[0:15]

    #index e escolher colunas
    #base = base.set_index(['Country'])
    base = base[['Country','Rank', 'Documents', 'Citable documents', 'Citations', 'Self-citations'
    , 'Citations per document', 'H index', 'Energy Supply', 'Energy Supply per Capita'
    , '% Renewable', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015']]
    
    base['avgGDP'] = base[['2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015']].mean(axis=1)
 
    #ordenar
    base = base.sort_values(by= ['avgGDP'], ascending = False)

    #selecionar o sexto maior
    base = base.iloc[5:6]
    
    #calcular a diferença
    GDP_2006 = base['2006']
    GDP_2015 = base['2015']
    GDP_dif = GDP_2015 - GDP_2006

    return GDP_dif

In [ ]:
answer_four()

3    2.467027e+11
dtype: float64


### Question 5
What is the mean `Energy Supply per Capita`?

*This function should return a single number.*

In [ ]:
def answer_five():

    import pandas as pd
    import numpy as np
    import math as mt

    #pd.set_option('max_rows', 99999)
    #pd.set_option('max_colwidth', 99999)
    #pd.describe_option('max_colwidth')

    #ler arquivo Energy Indicators.xls, já excluindo cabeçalho e rodapé
    energy = pd.read_excel("Energy_Indicators.xls", sheet_name="Energy", skiprows = 16, nrows = 228)
    
    #excluir primeiras colunas
    energy = energy.rename(columns = {'Unnamed: 2':'Country'})
    energy.drop(energy.columns[energy.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)

    #renomear colunas
    energy.columns = ['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable']

    #converter Energy Supply para gigajoules
    energy['Energy Supply'] = energy['Energy Supply'].apply(pd.to_numeric, errors='coerce')
    energy['Energy Supply'] = energy['Energy Supply']*1000000

    #renomear países
    energy['Country'] = energy['Country'].str.replace('\d+', '')
    energy = energy.replace(to_replace = 'Republic of Korea', value = 'South Korea')
    energy = energy.replace(to_replace = 'United States of America', value = 'United States')
    energy = energy.replace(to_replace = 'United Kingdom of Great Britain and Northern Ireland', value = 'United Kingdom')
    energy = energy.replace(to_replace = 'China, Hong Kong Special Administrative Region', value = 'Hong Kong')
    energy = energy.replace(to_replace = 'Bolivia (Plurinational State of)', value = 'Bolivia')
    energy = energy.replace(to_replace = 'Falkland Islands (Malvinas)', value = 'Falkland Islands')
    energy = energy.replace(to_replace = 'Iran (Islamic Republic of)', value = 'Iran')
    energy = energy.replace(to_replace = 'Micronesia (Federated States of)', value = 'Micronesia')
    energy = energy.replace(to_replace = 'Sint Maarten (Dutch part)', value = 'Sint Maarten')
    energy = energy.replace(to_replace = 'Venezuela (Bolivarian Republic of)', value = 'Venezuela')

    #ler arquivo world_bank.csv, já excluindo cabeçalho
    GDP = pd.read_csv('world_bank.csv', sep=",", header = 4)

    #renomear países
    GDP = GDP.replace(to_replace = 'Korea, Rep.', value = 'South Korea')
    GDP = GDP.replace(to_replace = 'Iran, Islamic Rep.', value = 'Iran')
    GDP = GDP.replace(to_replace = 'Hong Kong SAR, China', value = 'Hong Kong')

    #ler arquivo scimagojr-3.xlsx
    ScimEn = pd.read_excel('scimagojr-3.xlsx')

    #somente os 15 primeiros países
    #ScimEn = ScimEn[0:15]

    #GDP entre 2006 e 2015
    GDP = GDP[['Country Name', 'Country Code', 'Indicator Name', 'Indicator Code'
        ,'2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015']]

    #mesclar
    base = pd.merge(ScimEn, energy, how = "left", left_on = "Country", right_on = "Country", indicator = False)
    base = pd.merge(base, GDP, how = "left", left_on = "Country", right_on = "Country Name", indicator = False)

    #somente os 15 primeiros
    base = base[0:15]

    #index e escolher colunas
    #base = base.set_index(['Country'])
    base = base[['Country','Rank', 'Documents', 'Citable documents', 'Citations', 'Self-citations'
    , 'Citations per document', 'H index', 'Energy Supply', 'Energy Supply per Capita'
    , '% Renewable', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015']]
    
    base['avgGDP'] = base[['2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015']].mean(axis=1)
 
    #média simples de Energy Supply per Capita:
    mean_energy_supply = base['Energy Supply per Capita'].mean(axis=0)

    return mean_energy_supply

In [ ]:
answer_five()

157.6


### Question 6
What country has the maximum % Renewable and what is the percentage?

*This function should return a tuple with the name of the country and the percentage.*

In [ ]:
def answer_six():

    import pandas as pd
    import numpy as np
    import math as mt

    #ler arquivo Energy Indicators.xls, já excluindo cabeçalho e rodapé
    energy = pd.read_excel("Energy_Indicators.xls", sheet_name="Energy", skiprows = 16, nrows = 228)
    
    #excluir primeiras colunas
    energy = energy.rename(columns = {'Unnamed: 2':'Country'})
    energy.drop(energy.columns[energy.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)

    #renomear colunas
    energy.columns = ['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable']

    #converter Energy Supply para gigajoules
    energy['Energy Supply'] = energy['Energy Supply'].apply(pd.to_numeric, errors='coerce')
    energy['Energy Supply'] = energy['Energy Supply']*1000000

    #renomear países
    energy['Country'] = energy['Country'].str.replace('\d+', '')
    energy = energy.replace(to_replace = 'Republic of Korea', value = 'South Korea')
    energy = energy.replace(to_replace = 'United States of America', value = 'United States')
    energy = energy.replace(to_replace = 'United Kingdom of Great Britain and Northern Ireland', value = 'United Kingdom')
    energy = energy.replace(to_replace = 'China, Hong Kong Special Administrative Region', value = 'Hong Kong')
    energy = energy.replace(to_replace = 'Bolivia (Plurinational State of)', value = 'Bolivia')
    energy = energy.replace(to_replace = 'Falkland Islands (Malvinas)', value = 'Falkland Islands')
    energy = energy.replace(to_replace = 'Iran (Islamic Republic of)', value = 'Iran')
    energy = energy.replace(to_replace = 'Micronesia (Federated States of)', value = 'Micronesia')
    energy = energy.replace(to_replace = 'Sint Maarten (Dutch part)', value = 'Sint Maarten')
    energy = energy.replace(to_replace = 'Venezuela (Bolivarian Republic of)', value = 'Venezuela')

    #ler arquivo world_bank.csv, já excluindo cabeçalho
    GDP = pd.read_csv('world_bank.csv', sep=",", header = 4)

    #renomear países
    GDP = GDP.replace(to_replace = 'Korea, Rep.', value = 'South Korea')
    GDP = GDP.replace(to_replace = 'Iran, Islamic Rep.', value = 'Iran')
    GDP = GDP.replace(to_replace = 'Hong Kong SAR, China', value = 'Hong Kong')

    #ler arquivo scimagojr-3.xlsx
    ScimEn = pd.read_excel('scimagojr-3.xlsx')

    #somente os 15 primeiros países
    #ScimEn = ScimEn[0:15]

    #GDP entre 2006 e 2015
    GDP = GDP[['Country Name', 'Country Code', 'Indicator Name', 'Indicator Code'
        ,'2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015']]

    #mesclar
    base = pd.merge(ScimEn, energy, how = "left", left_on = "Country", right_on = "Country", indicator = False)
    base = pd.merge(base, GDP, how = "left", left_on = "Country", right_on = "Country Name", indicator = False)

    #somente os 15 primeiros
    base = base[0:15]

    #index e escolher colunas
    #base = base.set_index(['Country'])
    base = base[['Country','Rank', 'Documents', 'Citable documents', 'Citations', 'Self-citations'
    , 'Citations per document', 'H index', 'Energy Supply', 'Energy Supply per Capita'
    , '% Renewable', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015']]
    
    base['avgGDP'] = base[['2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015']].mean(axis=1)
 
    #pegar a maior % Renewable
    base = base.sort_values(by= ['% Renewable'], ascending = False)
    base = base[0:1]
    base = base[['Country', '% Renewable']]

    #transformar em tuple
    tuple_renewable = tuple(base.itertuples(index=False, name=None))

    return tuple_renewable

In [ ]:
answer_six()

(('Brazil', 69.64803),)


### Question 7 
Create a new column that is the ratio of Self-Citations to Total Citations. 
What is the maximum value for this new column, and what country has the highest ratio?

*This function should return a tuple with the name of the country and the ratio.*

In [ ]:
def answer_seven():

    import pandas as pd
    import numpy as np
    import math as mt

    #ler arquivo Energy Indicators.xls, já excluindo cabeçalho e rodapé
    energy = pd.read_excel("Energy_Indicators.xls", sheet_name="Energy", skiprows = 16, nrows = 228)
    
    #excluir primeiras colunas
    energy = energy.rename(columns = {'Unnamed: 2':'Country'})
    energy.drop(energy.columns[energy.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)

    #renomear colunas
    energy.columns = ['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable']

    #converter Energy Supply para gigajoules
    energy['Energy Supply'] = energy['Energy Supply'].apply(pd.to_numeric, errors='coerce')
    energy['Energy Supply'] = energy['Energy Supply']*1000000

    #renomear países
    energy['Country'] = energy['Country'].str.replace('\d+', '')
    energy = energy.replace(to_replace = 'Republic of Korea', value = 'South Korea')
    energy = energy.replace(to_replace = 'United States of America', value = 'United States')
    energy = energy.replace(to_replace = 'United Kingdom of Great Britain and Northern Ireland', value = 'United Kingdom')
    energy = energy.replace(to_replace = 'China, Hong Kong Special Administrative Region', value = 'Hong Kong')
    energy = energy.replace(to_replace = 'Bolivia (Plurinational State of)', value = 'Bolivia')
    energy = energy.replace(to_replace = 'Falkland Islands (Malvinas)', value = 'Falkland Islands')
    energy = energy.replace(to_replace = 'Iran (Islamic Republic of)', value = 'Iran')
    energy = energy.replace(to_replace = 'Micronesia (Federated States of)', value = 'Micronesia')
    energy = energy.replace(to_replace = 'Sint Maarten (Dutch part)', value = 'Sint Maarten')
    energy = energy.replace(to_replace = 'Venezuela (Bolivarian Republic of)', value = 'Venezuela')

    #ler arquivo world_bank.csv, já excluindo cabeçalho
    GDP = pd.read_csv('world_bank.csv', sep=",", header = 4)

    #renomear países
    GDP = GDP.replace(to_replace = 'Korea, Rep.', value = 'South Korea')
    GDP = GDP.replace(to_replace = 'Iran, Islamic Rep.', value = 'Iran')
    GDP = GDP.replace(to_replace = 'Hong Kong SAR, China', value = 'Hong Kong')

    #ler arquivo scimagojr-3.xlsx
    ScimEn = pd.read_excel('scimagojr-3.xlsx')

    #somente os 15 primeiros países
    #ScimEn = ScimEn[0:15]

    #GDP entre 2006 e 2015
    GDP = GDP[['Country Name', 'Country Code', 'Indicator Name', 'Indicator Code'
        ,'2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015']]

    #mesclar
    base = pd.merge(ScimEn, energy, how = "left", left_on = "Country", right_on = "Country", indicator = False)
    base = pd.merge(base, GDP, how = "left", left_on = "Country", right_on = "Country Name", indicator = False)

    #somente os 15 primeiros
    base = base[0:15]

    #index e escolher colunas
    #base = base.set_index(['Country'])
    base = base[['Country','Rank', 'Documents', 'Citable documents', 'Citations', 'Self-citations'
    , 'Citations per document', 'H index', 'Energy Supply', 'Energy Supply per Capita'
    , '% Renewable', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015']]
    
    base['avgGDP'] = base[['2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015']].mean(axis=1)

    base['ratio'] = base['Self-citations']/base['Citations']

    base = base[['Country','ratio']]

    base = base.sort_values(by= ['ratio'], ascending = False)
    base = base[0:1]

    #transformar em tuple
    tuple_ratio = tuple(base.itertuples(index=False, name=None))

    return tuple_ratio

In [ ]:
answer_seven()

(('China', 0.6893126179389422),)


### Question 8

Create a column that estimates the population using Energy Supply and Energy Supply per capita. 
What is the third most populous country according to this estimate?

*This function should return a single string value.*

In [ ]:
def answer_eight():

    import pandas as pd
    import numpy as np
    import math as mt

    #ler arquivo Energy Indicators.xls, já excluindo cabeçalho e rodapé
    energy = pd.read_excel("Energy_Indicators.xls", sheet_name="Energy", skiprows = 16, nrows = 228)
    
    #excluir primeiras colunas
    energy = energy.rename(columns = {'Unnamed: 2':'Country'})
    energy.drop(energy.columns[energy.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)

    #renomear colunas
    energy.columns = ['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable']

    #converter Energy Supply para gigajoules
    energy['Energy Supply'] = energy['Energy Supply'].apply(pd.to_numeric, errors='coerce')
    energy['Energy Supply'] = energy['Energy Supply']*1000000

    #renomear países
    energy['Country'] = energy['Country'].str.replace('\d+', '')
    energy = energy.replace(to_replace = 'Republic of Korea', value = 'South Korea')
    energy = energy.replace(to_replace = 'United States of America', value = 'United States')
    energy = energy.replace(to_replace = 'United Kingdom of Great Britain and Northern Ireland', value = 'United Kingdom')
    energy = energy.replace(to_replace = 'China, Hong Kong Special Administrative Region', value = 'Hong Kong')
    energy = energy.replace(to_replace = 'Bolivia (Plurinational State of)', value = 'Bolivia')
    energy = energy.replace(to_replace = 'Falkland Islands (Malvinas)', value = 'Falkland Islands')
    energy = energy.replace(to_replace = 'Iran (Islamic Republic of)', value = 'Iran')
    energy = energy.replace(to_replace = 'Micronesia (Federated States of)', value = 'Micronesia')
    energy = energy.replace(to_replace = 'Sint Maarten (Dutch part)', value = 'Sint Maarten')
    energy = energy.replace(to_replace = 'Venezuela (Bolivarian Republic of)', value = 'Venezuela')

    #ler arquivo world_bank.csv, já excluindo cabeçalho
    GDP = pd.read_csv('world_bank.csv', sep=",", header = 4)

    #renomear países
    GDP = GDP.replace(to_replace = 'Korea, Rep.', value = 'South Korea')
    GDP = GDP.replace(to_replace = 'Iran, Islamic Rep.', value = 'Iran')
    GDP = GDP.replace(to_replace = 'Hong Kong SAR, China', value = 'Hong Kong')

    #ler arquivo scimagojr-3.xlsx
    ScimEn = pd.read_excel('scimagojr-3.xlsx')

    #somente os 15 primeiros países
    #ScimEn = ScimEn[0:15]

    #GDP entre 2006 e 2015
    GDP = GDP[['Country Name', 'Country Code', 'Indicator Name', 'Indicator Code'
        ,'2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015']]

    #mesclar
    base = pd.merge(ScimEn, energy, how = "left", left_on = "Country", right_on = "Country", indicator = False)
    base = pd.merge(base, GDP, how = "left", left_on = "Country", right_on = "Country Name", indicator = False)

    #somente os 15 primeiros
    base = base[0:15]

    #index e escolher colunas
    #base = base.set_index(['Country'])
    base = base[['Country','Rank', 'Documents', 'Citable documents', 'Citations', 'Self-citations'
    , 'Citations per document', 'H index', 'Energy Supply', 'Energy Supply per Capita'
    , '% Renewable', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015']]
    
    base['avgGDP'] = base[['2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015']].mean(axis=1)

    base['population'] = base['Energy Supply']/base['Energy Supply per Capita']

    base = base[2:3]

    #transformar em string
    terceiro_populoso = base['Country'].to_string(index = False)

    return terceiro_populoso

In [ ]:
answer_eight()

 Japan


### Question 9
Create a column that estimates the number of citable documents per person. 
What is the correlation between the number of citable documents per capita and the energy supply per capita? Use the `.corr()` method, (Pearson's correlation).

*This function should return a single number.*


In [ ]:
def answer_nine():

    import pandas as pd
    import numpy as np
    import math as mt

    #ler arquivo Energy Indicators.xls, já excluindo cabeçalho e rodapé
    energy = pd.read_excel("Energy_Indicators.xls", sheet_name="Energy", skiprows = 16, nrows = 228)
    
    #excluir primeiras colunas
    energy = energy.rename(columns = {'Unnamed: 2':'Country'})
    energy.drop(energy.columns[energy.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)

    #renomear colunas
    energy.columns = ['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable']

    #converter Energy Supply para gigajoules
    energy['Energy Supply'] = energy['Energy Supply'].apply(pd.to_numeric, errors='coerce')
    energy['Energy Supply'] = energy['Energy Supply']*1000000

    #renomear países
    energy['Country'] = energy['Country'].str.replace('\d+', '')
    energy = energy.replace(to_replace = 'Republic of Korea', value = 'South Korea')
    energy = energy.replace(to_replace = 'United States of America', value = 'United States')
    energy = energy.replace(to_replace = 'United Kingdom of Great Britain and Northern Ireland', value = 'United Kingdom')
    energy = energy.replace(to_replace = 'China, Hong Kong Special Administrative Region', value = 'Hong Kong')
    energy = energy.replace(to_replace = 'Bolivia (Plurinational State of)', value = 'Bolivia')
    energy = energy.replace(to_replace = 'Falkland Islands (Malvinas)', value = 'Falkland Islands')
    energy = energy.replace(to_replace = 'Iran (Islamic Republic of)', value = 'Iran')
    energy = energy.replace(to_replace = 'Micronesia (Federated States of)', value = 'Micronesia')
    energy = energy.replace(to_replace = 'Sint Maarten (Dutch part)', value = 'Sint Maarten')
    energy = energy.replace(to_replace = 'Venezuela (Bolivarian Republic of)', value = 'Venezuela')

    #ler arquivo world_bank.csv, já excluindo cabeçalho
    GDP = pd.read_csv('world_bank.csv', sep=",", header = 4)

    #renomear países
    GDP = GDP.replace(to_replace = 'Korea, Rep.', value = 'South Korea')
    GDP = GDP.replace(to_replace = 'Iran, Islamic Rep.', value = 'Iran')
    GDP = GDP.replace(to_replace = 'Hong Kong SAR, China', value = 'Hong Kong')

    #ler arquivo scimagojr-3.xlsx
    ScimEn = pd.read_excel('scimagojr-3.xlsx')

    #somente os 15 primeiros países
    #ScimEn = ScimEn[0:15]

    #GDP entre 2006 e 2015
    GDP = GDP[['Country Name', 'Country Code', 'Indicator Name', 'Indicator Code'
        ,'2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015']]

    #mesclar
    base = pd.merge(ScimEn, energy, how = "left", left_on = "Country", right_on = "Country", indicator = False)
    base = pd.merge(base, GDP, how = "left", left_on = "Country", right_on = "Country Name", indicator = False)

    #somente os 15 primeiros
    base = base[0:15]

    #index e escolher colunas
    #base = base.set_index(['Country'])
    base = base[['Country','Rank', 'Documents', 'Citable documents', 'Citations', 'Self-citations'
    , 'Citations per document', 'H index', 'Energy Supply', 'Energy Supply per Capita'
    , '% Renewable', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015']]
    
    base['avgGDP'] = base[['2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015']].mean(axis=1)

    base['population'] = base['Energy Supply']/base['Energy Supply per Capita']

    #citable documents per person
    base['ratio_citable'] = base['Citable documents']/base['population']

    #correlação
    base['Energy Supply per Capita'] = base['Energy Supply per Capita'].astype(float)
    base = base[['ratio_citable', 'Energy Supply per Capita']]
    correl = base.corr(method = 'pearson')
    correl = np.float64(correl)
    correl = correl.item()
  
    return correl

In [ ]:
answer_nine()

1.0


### Question 10
Create a new column with a 1 if the country's % Renewable value is at or above the median for all countries in the top 15, and a 0 if the country's % Renewable value is below the median.

*This function should return a series named `HighRenew` whose index is the country name sorted in ascending order of rank.*

In [ ]:
def answer_ten():

    import pandas as pd
    import numpy as np
    import math as mt

    #ler arquivo Energy Indicators.xls, já excluindo cabeçalho e rodapé
    energy = pd.read_excel("Energy_Indicators.xls", sheet_name="Energy", skiprows = 16, nrows = 228)
    
    #excluir primeiras colunas
    energy = energy.rename(columns = {'Unnamed: 2':'Country'})
    energy.drop(energy.columns[energy.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)

    #renomear colunas
    energy.columns = ['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable']

    #converter Energy Supply para gigajoules
    energy['Energy Supply'] = energy['Energy Supply'].apply(pd.to_numeric, errors='coerce')
    energy['Energy Supply'] = energy['Energy Supply']*1000000

    #renomear países
    energy['Country'] = energy['Country'].str.replace('\d+', '')
    energy = energy.replace(to_replace = 'Republic of Korea', value = 'South Korea')
    energy = energy.replace(to_replace = 'United States of America', value = 'United States')
    energy = energy.replace(to_replace = 'United Kingdom of Great Britain and Northern Ireland', value = 'United Kingdom')
    energy = energy.replace(to_replace = 'China, Hong Kong Special Administrative Region', value = 'Hong Kong')
    energy = energy.replace(to_replace = 'Bolivia (Plurinational State of)', value = 'Bolivia')
    energy = energy.replace(to_replace = 'Falkland Islands (Malvinas)', value = 'Falkland Islands')
    energy = energy.replace(to_replace = 'Iran (Islamic Republic of)', value = 'Iran')
    energy = energy.replace(to_replace = 'Micronesia (Federated States of)', value = 'Micronesia')
    energy = energy.replace(to_replace = 'Sint Maarten (Dutch part)', value = 'Sint Maarten')
    energy = energy.replace(to_replace = 'Venezuela (Bolivarian Republic of)', value = 'Venezuela')

    #ler arquivo world_bank.csv, já excluindo cabeçalho
    GDP = pd.read_csv('world_bank.csv', sep=",", header = 4)

    #renomear países
    GDP = GDP.replace(to_replace = 'Korea, Rep.', value = 'South Korea')
    GDP = GDP.replace(to_replace = 'Iran, Islamic Rep.', value = 'Iran')
    GDP = GDP.replace(to_replace = 'Hong Kong SAR, China', value = 'Hong Kong')

    #ler arquivo scimagojr-3.xlsx
    ScimEn = pd.read_excel('scimagojr-3.xlsx')

    #somente os 15 primeiros países
    #ScimEn = ScimEn[0:15]

    #GDP entre 2006 e 2015
    GDP = GDP[['Country Name', 'Country Code', 'Indicator Name', 'Indicator Code'
        ,'2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015']]

    #mesclar
    base = pd.merge(ScimEn, energy, how = "left", left_on = "Country", right_on = "Country", indicator = False)
    base = pd.merge(base, GDP, how = "left", left_on = "Country", right_on = "Country Name", indicator = False)

    #somente os 15 primeiros
    base = base[0:15]

    #index e escolher colunas
    #base = base.set_index(['Country'])
    base = base[['Country','Rank', 'Documents', 'Citable documents', 'Citations', 'Self-citations'
    , 'Citations per document', 'H index', 'Energy Supply', 'Energy Supply per Capita'
    , '% Renewable', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015']]
    
    #calculo da mediana
    base['median_renewable'] = (base['% Renewable'].gt(base['% Renewable'].median())).astype(int)

    #ordenar por rank
    #base = base.sort_values(by= ['Rank'], ascending = False)

    #série HighRenew
    HighRenew =  pd.Series(base['median_renewable'].values, index = base['Country'], name = 'HighRenew')

    return HighRenew

In [ ]:
answer_ten()

Country
China                 1
United States         0
Japan                 0
United Kingdom        0
Russian Federation    1
Canada                1
Germany               1
India                 0
France                0
South Korea           0
Italy                 1
Spain                 1
Iran                  0
Australia             0
Brazil                1
Name: HighRenew, dtype: int64


### Question 11
Use the following dictionary to group the Countries by Continent, then create a dateframe that displays the sample size (the number of countries in each continent bin), and the sum, mean, and std deviation for the estimated population of each country.

```python
ContinentDict  = {'China':'Asia', 
                  'United States':'North America', 
                  'Japan':'Asia', 
                  'United Kingdom':'Europe', 
                  'Russian Federation':'Europe', 
                  'Canada':'North America', 
                  'Germany':'Europe', 
                  'India':'Asia',
                  'France':'Europe', 
                  'South Korea':'Asia', 
                  'Italy':'Europe', 
                  'Spain':'Europe', 
                  'Iran':'Asia',
                  'Australia':'Australia', 
                  'Brazil':'South America'}
```

*This function should return a DataFrame with index named Continent `['Asia', 'Australia', 'Europe', 'North America', 'South America']` and columns `['size', 'sum', 'mean', 'std']`*

In [ ]:
def answer_eleven():

    import pandas as pd
    import numpy as np
    import math as mt

    #ler arquivo Energy Indicators.xls, já excluindo cabeçalho e rodapé
    energy = pd.read_excel("Energy_Indicators.xls", sheet_name="Energy", skiprows = 16, nrows = 228)
    
    #excluir primeiras colunas
    energy = energy.rename(columns = {'Unnamed: 2':'Country'})
    energy.drop(energy.columns[energy.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)

    #renomear colunas
    energy.columns = ['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable']

    #converter Energy Supply para gigajoules
    energy['Energy Supply'] = energy['Energy Supply'].apply(pd.to_numeric, errors='coerce')
    energy['Energy Supply'] = energy['Energy Supply']*1000000

    #renomear países
    energy['Country'] = energy['Country'].str.replace('\d+', '')
    energy = energy.replace(to_replace = 'Republic of Korea', value = 'South Korea')
    energy = energy.replace(to_replace = 'United States of America', value = 'United States')
    energy = energy.replace(to_replace = 'United Kingdom of Great Britain and Northern Ireland', value = 'United Kingdom')
    energy = energy.replace(to_replace = 'China, Hong Kong Special Administrative Region', value = 'Hong Kong')
    energy = energy.replace(to_replace = 'Bolivia (Plurinational State of)', value = 'Bolivia')
    energy = energy.replace(to_replace = 'Falkland Islands (Malvinas)', value = 'Falkland Islands')
    energy = energy.replace(to_replace = 'Iran (Islamic Republic of)', value = 'Iran')
    energy = energy.replace(to_replace = 'Micronesia (Federated States of)', value = 'Micronesia')
    energy = energy.replace(to_replace = 'Sint Maarten (Dutch part)', value = 'Sint Maarten')
    energy = energy.replace(to_replace = 'Venezuela (Bolivarian Republic of)', value = 'Venezuela')

    #ler arquivo world_bank.csv, já excluindo cabeçalho
    GDP = pd.read_csv('world_bank.csv', sep=",", header = 4)

    #renomear países
    GDP = GDP.replace(to_replace = 'Korea, Rep.', value = 'South Korea')
    GDP = GDP.replace(to_replace = 'Iran, Islamic Rep.', value = 'Iran')
    GDP = GDP.replace(to_replace = 'Hong Kong SAR, China', value = 'Hong Kong')

    #ler arquivo scimagojr-3.xlsx
    ScimEn = pd.read_excel('scimagojr-3.xlsx')

    #GDP entre 2006 e 2015
    GDP = GDP[['Country Name', 'Country Code', 'Indicator Name', 'Indicator Code'
        ,'2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015']]

    #mesclar
    base = pd.merge(ScimEn, energy, how = "left", left_on = "Country", right_on = "Country", indicator = False)
    base = pd.merge(base, GDP, how = "left", left_on = "Country", right_on = "Country Name", indicator = False)

    #somente os 15 primeiros
    base = base[0:15]

    #index e escolher colunas
    base = base.set_index(['Country'])
    base = base[['Rank', 'Documents', 'Citable documents', 'Citations', 'Self-citations'
    , 'Citations per document', 'H index', 'Energy Supply', 'Energy Supply per Capita'
    , '% Renewable', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015']]

    #dicionário
    ContinentDict  = {'China':'Asia', 
                      'United States':'North America', 
                      'Japan':'Asia', 
                      'United Kingdom':'Europe', 
                      'Russian Federation':'Europe', 
                      'Canada':'North America', 
                      'Germany':'Europe', 
                      'India':'Asia',
                      'France':'Europe', 
                      'South Korea':'Asia', 
                      'Italy':'Europe', 
                      'Spain':'Europe', 
                      'Iran':'Asia',
                      'Australia':'Australia', 
                      'Brazil':'South America'}
    
    #criar novas colunas
    #base['size'] = None
    #base['continent'] = None

    agrupamentos = pd.DataFrame(columns=['size', 'sum', 'mean', 'std'])
    base['population'] = base['Energy Supply']/base['Energy Supply per Capita']

    for continent, count in base.groupby(ContinentDict):
        agrupamentos.loc[continent] = [len(count), count['population'].sum(
        ), count['population'].mean(), count['population'].std()]

    return agrupamentos

In [ ]:
answer_eleven()

               size           sum          mean           std
Asia            5.0  2.898666e+09  5.797333e+08  6.790979e+08
Australia       1.0  2.331602e+07  2.331602e+07           NaN
Europe          6.0  4.579297e+08  7.632161e+07  3.464767e+07
North America   2.0  3.528552e+08  1.764276e+08  1.996696e+08
South America   1.0  2.059153e+08  2.059153e+08           NaN


,size,sum,mean,std
Asia,5.0,2.898666e+09,5.797333e+08,6.790979e+08
Australia,1.0,2.331602e+07,2.331602e+07,NaN
Europe,6.0,4.579297e+08,7.632161e+07,3.464767e+07
North America,2.0,3.528552e+08,1.764276e+08,1.996696e+08
South America,1.0,2.059153e+08,2.059153e+08,NaN


### Question 12
Cut % Renewable into 5 bins. Group Top15 by the Continent, as well as these new % Renewable bins. How many countries are in each of these groups?

*This function should return a __Series__ with a MultiIndex of `Continent`, then the bins for `% Renewable`. Do not include groups with no countries.*

In [ ]:
def answer_twelve():
    import pandas as pd
    import numpy as np
    import math as mt

    #ler arquivo Energy Indicators.xls, já excluindo cabeçalho e rodapé
    energy = pd.read_excel("Energy_Indicators.xls", sheet_name="Energy", skiprows = 16, nrows = 228)
    
    #excluir primeiras colunas
    energy = energy.rename(columns = {'Unnamed: 2':'Country'})
    energy.drop(energy.columns[energy.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)

    #renomear colunas
    energy.columns = ['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable']

    #converter Energy Supply para gigajoules
    energy['Energy Supply'] = energy['Energy Supply'].apply(pd.to_numeric, errors='coerce')
    energy['Energy Supply'] = energy['Energy Supply']*1000000

    #renomear países
    energy['Country'] = energy['Country'].str.replace('\d+', '')
    energy = energy.replace(to_replace = 'Republic of Korea', value = 'South Korea')
    energy = energy.replace(to_replace = 'United States of America', value = 'United States')
    energy = energy.replace(to_replace = 'United Kingdom of Great Britain and Northern Ireland', value = 'United Kingdom')
    energy = energy.replace(to_replace = 'China, Hong Kong Special Administrative Region', value = 'Hong Kong')
    energy = energy.replace(to_replace = 'Bolivia (Plurinational State of)', value = 'Bolivia')
    energy = energy.replace(to_replace = 'Falkland Islands (Malvinas)', value = 'Falkland Islands')
    energy = energy.replace(to_replace = 'Iran (Islamic Republic of)', value = 'Iran')
    energy = energy.replace(to_replace = 'Micronesia (Federated States of)', value = 'Micronesia')
    energy = energy.replace(to_replace = 'Sint Maarten (Dutch part)', value = 'Sint Maarten')
    energy = energy.replace(to_replace = 'Venezuela (Bolivarian Republic of)', value = 'Venezuela')

    #ler arquivo world_bank.csv, já excluindo cabeçalho
    GDP = pd.read_csv('world_bank.csv', sep=",", header = 4)

    #renomear países
    GDP = GDP.replace(to_replace = 'Korea, Rep.', value = 'South Korea')
    GDP = GDP.replace(to_replace = 'Iran, Islamic Rep.', value = 'Iran')
    GDP = GDP.replace(to_replace = 'Hong Kong SAR, China', value = 'Hong Kong')

    #ler arquivo scimagojr-3.xlsx
    ScimEn = pd.read_excel('scimagojr-3.xlsx')

    #GDP entre 2006 e 2015
    GDP = GDP[['Country Name', 'Country Code', 'Indicator Name', 'Indicator Code'
        ,'2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015']]

    #mesclar
    base = pd.merge(ScimEn, energy, how = "left", left_on = "Country", right_on = "Country", indicator = False)
    base = pd.merge(base, GDP, how = "left", left_on = "Country", right_on = "Country Name", indicator = False)

    #somente os 15 primeiros
    base = base[0:15]

    #index e escolher colunas
    base = base.set_index(['Country'])
    base = base[['Rank', 'Documents', 'Citable documents', 'Citations', 'Self-citations'
    , 'Citations per document', 'H index', 'Energy Supply', 'Energy Supply per Capita'
    , '% Renewable', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015']]

    #dicionário
    ContinentDict = {'China': 'Asia',
                     'United States': 'North America',
                     'Japan': 'Asia',
                     'United Kingdom': 'Europe',
                     'Russian Federation': 'Europe',
                     'Canada': 'North America',
                     'Germany': 'Europe',
                     'India': 'Asia',
                     'France': 'Europe',
                     'South Korea': 'Asia',
                     'Italy': 'Europe',
                     'Spain': 'Europe',
                     'Iran': 'Asia',
                     'Australia': 'Australia',
                     'Brazil': 'South America'
                     }
    
    #criar index de continentes
    base['Continent'] = base.index.map(lambda c: ContinentDict[c])
    
    #criar bins
    base['% Renewable bins'] = pd.cut(base['% Renewable'], 5)

    #agrupar e remover os que não tem país nenhum
    paises = base.groupby(['Continent', '% Renewable bins']).size()
    paises.name = 'paises'
    paises = paises[paises != 0]
    
    return paises

In [ ]:
answer_twelve()

Continent      % Renewable bins
Asia           (2.212, 15.753]     4
               (15.753, 29.227]    1
Australia      (2.212, 15.753]     1
Europe         (2.212, 15.753]     1
               (15.753, 29.227]    3
               (29.227, 42.701]    2
North America  (2.212, 15.753]     1
               (56.174, 69.648]    1
South America  (56.174, 69.648]    1
Name: paises, dtype: int64

### Question 13
Convert the Population Estimate series to a string with thousands separator (using commas). Do not round the results.

e.g. 317615384.61538464 -> 317,615,384.61538464

*This function should return a Series `PopEst` whose index is the country name and whose values are the population estimate string.*

In [ ]:
def answer_thirteen():

    import pandas as pd
    import numpy as np
    import math as mt

    #ler arquivo Energy Indicators.xls, já excluindo cabeçalho e rodapé
    energy = pd.read_excel("Energy_Indicators.xls", sheet_name="Energy", skiprows = 16, nrows = 228)
    
    #excluir primeiras colunas
    energy = energy.rename(columns = {'Unnamed: 2':'Country'})
    energy.drop(energy.columns[energy.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)

    #renomear colunas
    energy.columns = ['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable']

    #converter Energy Supply para gigajoules
    energy['Energy Supply'] = energy['Energy Supply'].apply(pd.to_numeric, errors='coerce')
    energy['Energy Supply'] = energy['Energy Supply']*1000000

    #renomear países
    energy['Country'] = energy['Country'].str.replace('\d+', '')
    energy = energy.replace(to_replace = 'Republic of Korea', value = 'South Korea')
    energy = energy.replace(to_replace = 'United States of America', value = 'United States')
    energy = energy.replace(to_replace = 'United Kingdom of Great Britain and Northern Ireland', value = 'United Kingdom')
    energy = energy.replace(to_replace = 'China, Hong Kong Special Administrative Region', value = 'Hong Kong')
    energy = energy.replace(to_replace = 'Bolivia (Plurinational State of)', value = 'Bolivia')
    energy = energy.replace(to_replace = 'Falkland Islands (Malvinas)', value = 'Falkland Islands')
    energy = energy.replace(to_replace = 'Iran (Islamic Republic of)', value = 'Iran')
    energy = energy.replace(to_replace = 'Micronesia (Federated States of)', value = 'Micronesia')
    energy = energy.replace(to_replace = 'Sint Maarten (Dutch part)', value = 'Sint Maarten')
    energy = energy.replace(to_replace = 'Venezuela (Bolivarian Republic of)', value = 'Venezuela')

    #ler arquivo world_bank.csv, já excluindo cabeçalho
    GDP = pd.read_csv('world_bank.csv', sep=",", header = 4)

    #renomear países
    GDP = GDP.replace(to_replace = 'Korea, Rep.', value = 'South Korea')
    GDP = GDP.replace(to_replace = 'Iran, Islamic Rep.', value = 'Iran')
    GDP = GDP.replace(to_replace = 'Hong Kong SAR, China', value = 'Hong Kong')

    #ler arquivo scimagojr-3.xlsx
    ScimEn = pd.read_excel('scimagojr-3.xlsx')

    #GDP entre 2006 e 2015
    GDP = GDP[['Country Name', 'Country Code', 'Indicator Name', 'Indicator Code'
        ,'2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015']]

    #mesclar
    base = pd.merge(ScimEn, energy, how = "left", left_on = "Country", right_on = "Country", indicator = False)
    base = pd.merge(base, GDP, how = "left", left_on = "Country", right_on = "Country Name", indicator = False)

    #somente os 15 primeiros
    base = base[0:15]

    #index e escolher colunas
    #base = base.set_index(['Country'])
    base = base[['Country','Rank', 'Documents', 'Citable documents', 'Citations', 'Self-citations'
    , 'Citations per document', 'H index', 'Energy Supply', 'Energy Supply per Capita'
    , '% Renewable', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015']]
    
    base['avgGDP'] = base[['2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015']].mean(axis=1)

    base['population'] = base['Energy Supply']/base['Energy Supply per Capita']

    #série PopEst 
    PopEst = pd.Series(base['population'].values, index = base['Country'], name = 'PopEst')
    PopEst = PopEst.apply(lambda x: '{:,}'.format(x))

    return PopEst

In [ ]:
answer_thirteen()

Country
China                 1,367,645,161.2903225
United States          317,615,384.61538464
Japan                  127,409,395.97315437
United Kingdom         63,870,967.741935484
Russian Federation            143,500,000.0
Canada                  35,239,864.86486486
Germany                 80,369,696.96969697
India                 1,276,730,769.2307692
France                  63,837,349.39759036
South Korea            49,805,429.864253394
Italy                  59,908,256.880733944
Spain                    46,443,396.2264151
Iran                    77,075,630.25210084
Australia              23,316,017.316017315
Brazil                 205,915,254.23728815
Name: PopEst, dtype: object